In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import os, sys
# base = "/Users/ckh/Documents/Poladrone/nb"
base = "D:/YoloV5_Hui/poladrone_nb"
sys.path.append(base + "/ref/pytorchYOLOv4")

from exp.nb_TrainingRunnner import *
from exp.nb_LoggingModule import custom_init_logger
from exp.nb_CustomDataLoader import Detection_dataset

In [13]:
from config.config import Cfg

In [14]:
data_list = "D:/YoloV5_Hui/Dataset/train2.txt"
val_data_list = "D:/YoloV5_Hui/Dataset/val2.txt"
# data_list = "/Users/ckh/OneDrive - Default Directory/Hui_Wan/train_npt.txt"

dataset = Detection_dataset(data_list, Cfg)
val_dataset = Detection_dataset(val_data_list, Cfg, val=True)

logging = custom_init_logger(log_dir='log', stdout=False)

log file path:log\log_2020-07-24_16-28-24.txt


In [15]:
dataset.id_names

{0: 'palm1_npt', 1: 'palm1_byt', 2: 'palm2_mtl'}

In [16]:
torch.cuda.is_available()

True

In [17]:
cfg = get_args(**Cfg)
cfg.classes = 1 ## hardcode num_classes

cfg.batch = 8
cfg.subdivisions = 2
cfg.TRAIN_EPOCHS = 3000

# os.environ['CUDA_VISIBLE_DEVICES']='1'
# torch.cuda.set_device(1)
os.environ["CUDA_VISIBLE_DEVICES"] = cfg.gpu

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')

print("Device:", device)

Device: cuda


In [18]:
model = Yolov4(cfg.pretrained,n_classes=cfg.classes)

if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)
model.to(device=device)

DataParallel(
  (module): Yolov4(
    (down1): DownSample1(
      (conv1): Conv_Bn_Activation(
        (conv): ModuleList(
          (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): Mish()
        )
      )
      (conv2): Conv_Bn_Activation(
        (conv): ModuleList(
          (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): Mish()
        )
      )
      (conv3): Conv_Bn_Activation(
        (conv): ModuleList(
          (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): Mish()
        )
      )
      (conv4): Conv_Bn_Activation(
        (conv): ModuleList(
          (0): Conv2d

In [10]:
def get_args(**kwargs):
    cfg = kwargs
    parser = argparse.ArgumentParser(description='Train the Model on images and target masks',
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    # parser.add_argument('-b', '--batch-size', metavar='B', type=int, nargs='?', default=2,
    #                     help='Batch size', dest='batchsize')
    parser.add_argument('-l', '--learning-rate', metavar='LR', type=float, nargs='?', default=0.001,
                        help='Learning rate', dest='learning_rate')
    parser.add_argument('-f', '--load', dest='load', type=str, default=None,
                        help='Load model from a .pth file')
    parser.add_argument('-g', '--gpu', metavar='G', type=str, default='-1',
                        help='GPU', dest='gpu')
    parser.add_argument('-dir', '--data-dir', type=str, default=None,
                        help='dataset dir', dest='dataset_dir')
    parser.add_argument('-pretrained', type=str, default=None, help='pretrained yolov4.conv.137')
    parser.add_argument('-classes', type=int, default=80, help='dataset classes')
    parser.add_argument('-train_label_path', dest='train_label', type=str, default='train.txt', help="train label path")
    parser.add_argument(
        '-optimizer', type=str, default='adam',
        help='training optimizer',
        dest='TRAIN_OPTIMIZER')
    parser.add_argument(
        '-iou-type', type=str, default='iou',
        help='iou type (iou, giou, diou, ciou)',
        dest='iou_type')
    parser.add_argument(
        '-keep-checkpoint-max', type=int, default=10,
        help='maximum number of checkpoints to keep. If set 0, all checkpoints will be kept',
        dest='keep_checkpoint_max')
    args = vars(parser.parse_args())

    # for k in args.keys():
    #     cfg[k] = args.get(k)
    cfg.update(args)

    return edict(cfg)

In [ ]:
import random
import torch
torch.manual_seed(20)
np.random.seed(20)
random.seed(20)


try:
    custom_train(
        train_dataset=dataset,
        val_dataset=val_dataset,
        model=model,
        config=cfg,
        epochs=cfg.TRAIN_EPOCHS,
        cp

        device=device,
        log_step=100)
except KeyboardInterrupt:
    torch.save(model.state_dict(), 'INTERRUPTED.pth')
    logging.info('Saved interrupt')
    try:
        sys.exit(0)
    except SystemExit:
        os._exit(0)

In [14]:
if __name__ == "__main__":
    print("Main")

Main
